In [1]:
## Estimación de viviendas para gobierno federal de Claudia Sheinbaum
##Borrar datos del entorno
rm(list=ls())

#Directorio para descargar de datos
#Esto se debe de cambiar en cada computadora
#setwd("D:/datos")
#creamos carpeta en donde almacenaremos los archivos
dir.create("envi", showWarnings=F)

In [2]:
# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr,showtext)
#Fuente para los gráficos
font_add_google("Poppins", "pop")
showtext_auto()

Loading required package: pacman



In [3]:
#Descarga de archivos====
url<-"https://www.inegi.org.mx/contenidos/programas/envi/2020/microdatos/envi_2020_base_de_datos_csv.zip"

temp <- tempfile()
download.file(url,
              mode = "wb",
              destfile = temp)
unzip(temp, exdir = "envi")
unlink(temp)

In [4]:
vivienda<-read.csv("envi/Bases de datos/TVIVIENDA.csv")%>%
  janitor::clean_names()%>%
    #renombrar
  rename(folio=1,
    tot_resid=p1_1,
    num_cuarto=p4_10a,
    mat_pared=p4_4,
    mat_pisos=p4_6,
    mat_techos=p4_5,
excusado=p4_11)%>%
    mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
#Calcular rezago habitacional
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    ))

In [33]:
#Leer demanda de vivienda
demanda<-read.csv("envi/Bases de datos/TDEMANDA.csv")%>%
  janitor::clean_names()%>%
  #renombrar
  rename(folio=1)%>%
#Pegar viviendas en rezago e ingresos
#left_join(vivirezago,by=c("folio","viv_sel"))%>%
#left_join(ingreso,by=c("folio","viv_sel","hogar"))%>%
###############################
#Criterios de discriminación####
###############################


#Filtrar aquellos en rezago
#filter(rezago=="En rezago")%>%
#Filtrar aquellos que tienen intención de comprar o construir
filter(p3a4 %in% c(2,3,4))%>%
#Discriminar por motivos
filter(p3a6 %in% c(5,3,6,7,8,4))%>%
mutate(tipo_vivi=case_when(
    p3a9 %in% c(1,2,3,4) ~ "Interés social",
    TRUE ~ "Otro tipo",

))%>%
#Discriminar por valor de la vivienda. Dejar solo aquellas que serán de interés social Menor a 1 millón 100 mil pesos
#filter(p3a9 %in% c(1,2,3,4))%>%

#Crear nombre de entidad
mutate(nom_ent=case_when(
ent==1~"Aguascalientes",
ent==2~"Baja California",
ent==3~"Baja California Sur",
ent==4~"Campeche",
ent==5~"Coahuila de Zaragoza",
ent==6~"Colima",
ent==7~"Chiapas",
ent==8~"Chihuahua",
ent==9~"Ciudad de México",
ent==10~"Durango",
ent==11~"Guanajuato",
ent==12~"Guerrero",
ent==13~"Hidalgo",
ent==14~"Jalisco",
ent==15~"México",
ent==16~"Michoacán de Ocampo",
ent==17~"Morelos",
ent==18~"Nayarit",
ent==19~"Nuevo León",
ent==20~"Oaxaca",
ent==21~"Puebla",
ent==22~"Querétaro",
ent==23~"Quintana Roo",
ent==24~"San Luis Potosí",
ent==25~"Sinaloa",
ent==26~"Sonora",
ent==27~"Tabasco",
ent==28~"Tamaulipas",
ent==29~"Tlaxcala",
ent==30~"Veracruz de Ignacio de la Llave",
ent==31~"Yucatán",
ent==32~"Zacatecas"))

In [34]:

options(survey.lonely.psu="certainty")

dism <-demanda%>%
  as_survey_design(ids=upm_dis,
                   strata=est_dis,
                   weights=factor)

In [35]:
dism%>%
filter(ent==9)%>%
summarise(viviendas=survey_total(vartype = "cv"))

viviendas,viviendas_cv
<dbl>,<dbl>
602647,0.04166585


In [39]:
dism%>%
filter(ent==9)%>%
group_by(tipo_vivi)%>%

summarise(viviendas=survey_total(vartype = "cv"),
          pct=survey_prop(vartype = "cv"))%>%
mutate(pct=round(pct*100,2))%>%

janitor::adorn_totals()

tipo_vivi,viviendas,viviendas_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Interés social,335045,0.06354218,55.6,0.05180677
Otro tipo,267602,0.08013776,44.4,0.06486349
Total,602647,0.14367994,100.0,0.11667026
